## Problem Statement

You will have to use PINNs to solve the fluid flow for the given geometry and flow parameters.

A 2D chip is placed inside a 2D channel. The flow enters inlet at $u=1.5\text{ m/s}$ and exits through the outlet which is a $0 Pa$. All the other walls are treated as no-slip. The kinematic viscosity $(\nu)$ for the flow is $0.02 \text{ }m^2/s$ and the density $(\rho)$ is $1 \text{ }kg/m^3$. The problem is shown in the figure below.

<img src="chip_2d_geom.png" alt="Drawing" style="width: 800px;"/>

## Challenge

The main challenge in this problem is to correctly formulate the problem using PINNs. In order to achieve that, you will have to complete the following parts successfully:
1. Define the correct geometry for the problem
2. Set-up the correct boundary conditions and equations
3. Create the neural network and solve the problem

A successful completion of the problem should result in distribution of flow variables as shown below. Also, you should aim to achieve a relative $L_2$ error of less than 0.2 for all the variables w.r.t the given OpenFOAM solution. 

<img src="challenge_results.png" alt="Drawing" style="width: 650px;"/>

In this template, we will have give you a skeleton code which you will fill in to define and solve the problem.

**Note: You need to edit the [`chip_2d_template.py`](../../source_code/chip_2d/chip_2d_template.py) script that is placed in the `chip_2d/` directory. The config files are already provide to you** 

Let us start with importing the required packages. Pay attention to the various modules and packages that are being imported, especially the equations, geometry and architectures. 

```python
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.utils.sympy.functions import parabola
from modulus.eq.pdes.navier_stokes import NavierStokes
from modulus.eq.pdes.basic import NormalDotVec
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    IntegralBoundaryConstraint,
)

from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
```

Now, define the Nodes for the computational graph, appropriate PDEs, simulation parameters and generate the geometry. You will be using the `NavierStokes` class from the PDES module for imposing the PDE constraints of this problem. 

```python
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    
    #TODO: Replace all the placeholders with appropriate values
    # make list of nodes to unroll graph on
    ns = NavierStokes(nu=0.02, rho=1.0, dim=2, time=False)
    normal_dot_vel = NormalDotVec(["u", "v"])
    flow_net = instantiate_arch(
        input_keys=[placeholder],
        output_keys=[placeholder],
        cfg=cfg.arch.fourier_net,
    )
    nodes = (
        ns.make_nodes()
        + normal_dot_vel.make_nodes()
        + [flow_net.make_node(name="flow_network", jit=cfg.jit)]
    )
```

Next, you will be using the 2D geometry modules for this example. Please fill in the appropriate values for each geometry. Remember, `Channel2D` and `Rectangle` are defined by its two endpoints. The difference between a channel and rectangle in Modulus is that, the channel geometry does not have bounding curves in the x-direction. This is helpful in getting a signed distance field that it infinite in x-direction. This can be important when the signed distance field is used as a wall distance in some of the turbulence models (Refer [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/text/foundational/zero_eq_turbulence.html) for more details). Hence we will create the inlet and outlet using `Line` geometry (*Please note that this is a 2d line, as opposed to the* `Line1D` *that was used in the diffusion tutorial*)

```python
    # add constraints to solver
    # simulation params
    channel_length = (-2.5, 2.5)
    channel_width = (-0.5, 0.5)
    chip_pos = -1.0
    chip_height = 0.6
    chip_width = 1.0
    inlet_vel = 1.5

    # define sympy variables to parametrize domain curves
    x, y = Symbol("x"), Symbol("y")
    
    #TODO: Replace x1, y1, x2, y2, and X's with appropriate values
    
    # define geometry
    channel = Channel2D(
        (channel_length[0], channel_width[0]), (channel_length[1], channel_width[1])
    )
    inlet = Line(
        (x1, y1),
        (x1, y2),
        normal=1,
    )
    outlet = Line(
        (x2, y1),
        (x2, y2),
        normal=1,
    )
    rec = Rectangle(
        (x1, y1),
        (x2, y2),
    )
    flow_rec = Rectangle(
        (chip_pos - 0.25, channel_width[0]),
        (chip_pos + chip_width + 0.25, channel_width[1]),
    )
    geo = channel - rec
    geo_hr = geo & flow_rec
    geo_lr = geo - flow_rec
```

The current problem is a channel flow with an incompressible fluid. In such cases, the mass flow rate through each cross-section of the channel and in turn the volumetric flow is constant. This can be used as an additional constraint in the problem which we will help us in improving the speed of convergence. 

Wherever, possible, using such additional knowledge about the problem can help in better and faster solutions. More examples of this can be found in the [Modulus User Documentation](https://docs.nvidia.com/deeplearning/modulus/text/foundational/scalar_transport.html).   

```python
    # Optional integral continuity planes to speed up convergence 
    x_pos = Symbol("x_pos")
    integral_line = Line((x_pos, channel_width[0]), (x_pos, channel_width[1]), 1)
    x_pos_range = {
        x_pos: lambda batch_size: np.full(
            (batch_size, 1), np.random.uniform(channel_length[0], channel_length[1])
        )
    }
```

Now you will use the created geometry to define the training constraints for the problem. Implement the required boundary conditions and equations below. Remember that you will have to create constraints for both for the boundary condition and to reduce the equation residuals. You can refer to the `NavierStokes` class from the PDEs module to check how the equations are defined and the keys required for each equation. 

Now use this understanding to define the problem. An example of the inlet boundary condition is shown. Also, the integral continuity constraint is already coded up for you. 

```python
    #TODO: Replace all the placeholders with appropriate values

    # make domain
    domain = Domain()

    # inlet
    inlet_parabola = parabola(y, channel_width[0], channel_width[1], inlet_vel)
    inlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=inlet,
        outvar={"u": inlet_parabola, "v": 0},
        batch_size=cfg.batch_size.inlet,
    )
    domain.add_constraint(inlet, "inlet")

    # outlet
    outlet = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=outlet,
        outvar={placeholder},
        batch_size=cfg.batch_size.outlet,
        criteria=Eq(x, channel_length[1]),
    )
    domain.add_constraint(outlet, "outlet")

    # no slip
    no_slip = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={placeholder},
        batch_size=cfg.batch_size.no_slip,
    )
    domain.add_constraint(no_slip, "no_slip")
    

    # interior lr
    interior_lr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_lr,
        outvar={placeholder},
        batch_size=cfg.batch_size.interior_lr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting={placeholder},
    )
    domain.add_constraint(interior_lr, "interior_lr")

    # interior hr
    interior_hr = PointwiseInteriorConstraint(
        nodes=nodes,
        geometry=geo_hr,
        outvar={placeholder},
        batch_size=cfg.batch_size.interior_hr,
        bounds={x: channel_length, y: channel_width},
        lambda_weighting={placeholder},
    )
    domain.add_constraint(interior_hr, "interior_hr")
    
    # integral continuity
    def integral_criteria(invar, params):
        sdf = geo.sdf(invar, params)
        return np.greater(sdf["sdf"], 0)
     
    # integral continuity
    integral_continuity = IntegralBoundaryConstraint(
        nodes=nodes,
        geometry=integral_line,
        outvar={"normal_dot_vel": 1},
        batch_size=cfg.batch_size.num_integral_continuity,
        integral_batch_size=cfg.batch_size.integral_continuity,
        lambda_weighting={"normal_dot_vel": 1},
        criteria=integral_criteria,
        parameterization=x_pos_range,
    )
    domain.add_constraint(integral_continuity, "integral_continuity")
```

Now, add validation data to the problem. The `openfoam` directory that contains the solution of same problem using OpenFOAM solver. The CSV file is read in and converted to a dictionary for you. Now, you will have to complete the definition of validators. 

```python
    #TODO: Set the appropriate normalization for the validation data
    # The validation data has domain extents of (0,0) to (5,1). Normalize this based on your definition of the domain

    # add validation data
    mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
    openfoam_var = csv_to_dict(to_absolute_path("openfoam/2D_chip_fluid0.csv"), mapping)
    openfoam_var["x"] -= placeholder  # TODO normalize pos
    openfoam_var["y"] -= placeholder  # TODO normalize pos
    openfoam_invar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["x", "y"]
    }
    openfoam_outvar_numpy = {
        key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
    }
    openfoam_validator = PointwiseValidator(
        nodes=nodes,
        invar=openfoam_invar_numpy,
        true_outvar=openfoam_outvar_numpy,
    )
    domain.add_validator(openfoam_validator)
```

Now, complete the last part of the code by creating the `Solver` to solve the problem. The important hyperparameters of the problem are defined for you in the config file. Feel free to tweak them and observe its behavior on the results and speed of convergence. 

```python
    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()
```

Once you have completed all the sections of the code in the template script ([`chip_2d_template.py`](../../source_code/chip_2d/chip_2d_template.py)), you can execute it as we have seen in the earlier tutorials. Record your relative L2 errors with respect to the OpenFOAM solution and try to achieve errors for all the variables lower than 0.2. Also try to visualize your results using contour plots by reading in the `.npz` files created in the network checkpoint. 



In [ ]:
!python3 ../../source_code/chip_2d/chip_2d_template.py

# Licensing
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).
